In [1]:
import numpy as np
import pandas as pd
import os
from human_eval.data import write_jsonl, read_problems, extract_python

In [4]:
df = pd.read_json("data/samples_codellama.jsonl", lines=True)
df["full_response"]

0      Here is the complete code:\n```python\nimport ...
1      Plan:\n\n1. Import the necessary libraries for...
2      [PYTHON]\nimport numpy as np\nfrom skimage imp...
3      Here is a possible implementation of the bland...
4      Here's the code for the function `combine_colu...
                             ...                        
565    Here is the complete code for the function `wo...
566    Plan:\n\n1. Convert the input image to graysca...
567    Here is the complete code for the function `wo...
568    ```python\nimport numpy as np\nimport pandas a...
569    Here is the complete code for the function `wo...
Name: full_response, Length: 570, dtype: object

In [24]:
directory = "data/"

collection = []
for filename in os.listdir(directory):
    jsonl_file = filename.endswith(".jsonl") and not filename.endswith("results.jsonl") and not filename.endswith("bia.jsonl")
    results_jsonl_file = filename.endswith("_results.jsonl")
    if jsonl_file:
        df = pd.read_json(directory + filename, lines=True)
        df['model'] = filename.replace(".jsonl", "").replace("samples_", "")
        collection.append(df)
    # if results_jsonl_file:
    #     df = pd.read_json(directory + filename, lines=True)
    #     df['model'] = filename.replace("_results.jsonl", "").replace("samples_", "")
    #     collection.append(df)

df = pd.concat(collection)


In [25]:
df

,task_id,completion,full_response,model
0,../test_cases/apply_otsu_threshold_and_count_p...,\n# 1. Import necessary libraries: OpenCV for ...,Here is the completed code with comments on ho...,llama3-8b-instruct-fp16
1,../test_cases/binary_closing.ipynb,"\nimport cv2\nfrom skimage import filters, mea...",Here is the plan as comments:\n\n# Plan:\n# 1....,llama3-8b-instruct-fp16
2,../test_cases/binary_skeleton.ipynb,Python\n# Plan:\n# 1. Import necessary librari...,Here is the complete code:\n\n```Python\n# Pla...,llama3-8b-instruct-fp16
3,../test_cases/bland_altman.ipynb,\n# Plan:\n# 1. Check if the input dataframe a...,Here is the complete code:\n\n```python\n# Pla...,llama3-8b-instruct-fp16
4,../test_cases/combine_columns_of_tables.ipynb,\n# Plan:\n# 1. Check if index column exists i...,Here is the complete code:\n\n```python\n# Pla...,llama3-8b-instruct-fp16
...,...,...,...,...
565,../test_cases/workflow_batch_process_folder_me...,1. Import the necessary libraries that will be...,1. Import the necessary libraries that will be...,codellama-70b-instruct-q4_0
566,../test_cases/workflow_segmentation_counting.i...,\nimport numpy as np # For working with arrays...,1. Import the necessary Python libraries:\n\n`...,codellama-70b-instruct-q4_0
567,../test_cases/workflow_segmentation_measuremen...,\nimport cv2 # For thresholding and image pro...,1. **Thresholding**: Using Otsu's thresholding...,codellama-70b-instruct-q4_0
568,../test_cases/workflow_segment_measure_umap.ipynb,"\nimport numpy as np\nfrom skimage import io, ...",```python\nimport numpy as np\nfrom skimage im...,codellama-70b-instruct-q4_0


In [21]:
test = """
hallo
```
code
```
bla
```
code2
```
"""
extract_python(test)

'\ncode\n\ncode2\n'

In [26]:
# Function-mapping to extract python code from full_response
tmp_dir = "tmp_data/"
if not os.path.exists(tmp_dir):
    os.mkdir(tmp_dir)

df["completion"] = df["full_response"].astype(str).apply(extract_python)
grouped = df.groupby("model")

for name, group in grouped:
    records = group.drop("model", axis=1).to_dict("records")
    write_jsonl(f"{tmp_dir}/samples_{name}.jsonl", records)

# Testing code after 
for filename in os.listdir("data"):
    jsonl_file = filename.endswith(".jsonl") and not filename.endswith("results.jsonl") and not filename.endswith("bia.jsonl")
    results_jsonl_file = filename.endswith("_results.jsonl")
    if jsonl_file:
        model_id = filename.split(".jsonl")[0]
        df_orig = pd.read_json(directory+filename, lines=True)
        df_new = pd.read_json(f"{tmp_dir}{model_id}.jsonl", lines=True)
        
        if not df_orig.equals(df_new):
            # checks inequalities
            num_diff = np.size(np.where(~np.all((df_orig == df_new).to_numpy(), axis=1)))
            print(f"{model_id} is different in {num_diff} entries")
        

samples_llama3-8b-instruct-fp16 is different in 248 entries
samples_llama3-70b-instruct-q4_0 is different in 64 entries
samples_llama3-70b-instruct-q8_0 is different in 94 entries
samples_command-r-plus-104b-q4_0 is different in 1 entries
